In [ ]:
# from openai import OpenAI
# from pydantic import BaseModel, Field
# from typing import List
# import os
# import instructor
# from rag.vector import ChromaRag

# class Character(BaseModel):
#     name: str
#     description: int
#     fact: List[str] = Field(..., description="A list of facts about the character")

# class Answer(BaseModel):
#     answer: str = Field(..., description="Exact requirement as per the user's query.")
#     references: List[str] = Field(..., description="Key points or facts pulled from the documents. Include Metadata if available.")

# client = instructor.from_openai(
#     OpenAI(
#         base_url="http://localhost:11434/v1",
#         api_key="ollama",  
#     ),
#     mode=instructor.Mode.JSON,
# )

# mdoel_embed = "mxbai-embed-large"
# model_chat = "llama3.1:8b"
# db_path = "markdown_db"
# source_dir = r"C:\Users\shive\OneDrive\Documents\DnD\Adventures\Stone-Heart Hollow"
# top_k = 5

# collection_name = os.path.basename(source_dir)
# chroma_rag = ChromaRag(
#         source_directory=source_dir,
#         collection_name=collection_name,
#         db_path=db_path,
#         model_name=mdoel_embed
#     )

# question = "What was in the letter sent to Councillor Bren?"
# docs, ids, metadata = chroma_rag.retrieve(question, k=top_k)
# docs_text = "\n---\n".join(docs)
# # print(docs)
# prompt = f"""
#     ==================
#     You are a virtual assistant for a Dungeon Master running Dungeons & Dragons (D&D) sessions.

#     - Helping prepare and run sessions by providing ideas, rules clarifications, encounter design, and narrative suggestions.

#     - Answering questions using provided campaign documents, focusing only on the relevant content.

#     - If no documents are provided, rely on your general D&D knowledge (primarily 5th Edition unless specified).

#     Be accurate, concise, and helpful. Prioritize clarity when referencing documents. Prioritize clarity and creativity when assisting with gameplay and storytelling.
#     ==================

#     ==================
#     DOCUMENT CONTEXT:
#     {docs_text}
#     ==================

#     ==================
#     Query:
#     {question}
#     ==================
# """

# resp = client.chat.completions.create(
#     model="gemma3",
#     messages=[
#         {
#             "role": "user",
#             "content": prompt,
#         }
#     ],
#     response_model=Answer,
# )
# print(resp.model_dump_json(indent=2))


In [1]:
from rag.vector import ChromaRag
from llm.responses import *
import os

model_embed = "mxbai-embed-large"
model_chat = "llama3.1:8b"
db_path = "markdown_db"
source_dir = r"C:\Users\shive\OneDrive\Documents\DnD\Adventures\Stone-Heart Hollow"
top_k = 5

collection_name = os.path.basename(source_dir)
chroma_rag = ChromaRag(
        source_directory=source_dir,
        collection_name=collection_name,
        db_path=db_path,
        model_name=model_embed
    )

instructor_assistant = InstructorAssistant(model=model_chat)

question = "What was in the letter sent to Councillor Bren?"
docs, ids, metadata = chroma_rag.retrieve(question, k=top_k)

response = instructor_assistant.ask(question=question, context=docs, response_model=Answer)
print(response.model_dump_json(indent=2))

{
  "answer": "The letter warned of judgement coming, saying that Councilor Bren needs to flee the Hollow or face a stone-cold gaze and an eternal fate.",
  "references": [
    "Letter from Kern to Councillor Bren",
    "'You sealed the wound, but not the rot beneath.' - The letter"
  ]
}


In [2]:
question = "Make a new town character for Gallowmere"
docs, ids, metadata = chroma_rag.retrieve(question, k=20)

response = instructor_assistant.ask(question=question, context=docs, response_model=NPC)
print(response.model_dump_json(indent=2))

{
  "name": "Old Fenya",
  "race": "halfling",
  "role": "retired miner and tavern owner",
  "description": "has a glass eye and a sharp tongue, owns the Mine Down Tavern where rumors of ‘statue-men’ and cursed stones swirl over sour ale",
  "motivation": "united the townspeople against Kern's activities in the quarry",
  "secret": "Fenya was involved in the initial suppression of the cult, but the authorities found no evidence"
}


In [ ]:
question = "Make 3 new town characters for Gallowmere"
docs, ids, metadata = chroma_rag.retrieve(question, k=20)

response = instructor_assistant.ask(question=question, context=docs, response_model=NPCList)
print(response.model_dump_json(indent=2))

InstructorRetryException: Failed to validate model NPCList: 1 validation error for NPCList
  Input should be a valid array [type=list_type, input_value={'$schema': 'https://json...'NPC', 'type': 'object'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/list_type

In [3]:
question = "Make a new location for the players to explore within Gallowmere"
docs, ids, metadata = chroma_rag.retrieve(question, k=20)

response = instructor_assistant.ask(question=question, context=docs, response_model=Location)
print(response.model_dump_json(indent=2))

{
  "name": "Ruins of the Ancient Quarry",
  "type": "Abandoned mine",
  "description": "A long-abandoned quarry deep in the woods, said to have been the site of a catastrophic collapse. The entrance is shrouded in mystery and danger.",
  "danger_level": 8,
  "rumors": [
    "Whispers of strange noises coming from within the ruin at night",
    "Hunters who ventured into the quarry never returned",
    "An ancient, petrified body was found near the entrance"
  ]
}


In [8]:
question = "Make an item within the Gallowmere Hollow that is mundane"
docs, ids, metadata = chroma_rag.retrieve(question, k=20)

response = instructor_assistant.ask(question=question, context=docs, response_model=Item)
print(response.model_dump_json(indent=2))

{
  "name": "Miner's Pick",
  "item_type": "tool",
  "rarity": "common",
  "description": "A worn miner's pick with a rusty head and wooden handle, used by miners to extract ore from the quarry.",
  "effect": ""
}


In [2]:
question = "Make a rumour about Councillor Bren"
docs, ids, metadata = chroma_rag.retrieve(question, k=20)

response = instructor_assistant.ask(question=question, context=docs, response_model=Rumour)
print(response.model_dump_json(indent=2))

{
  "text": "Some say Councillor Bren is hiding more than just the truth – he's also hiding from it.",
  "truthfulness": "Rumor",
  "source": "Whispers from around town, but nothing concrete can be found..."
}


In [5]:
question = "Make a random name for me"
docs, ids, metadata = chroma_rag.retrieve(question, k=20)

response = instructor_assistant.ask(question=question, context=docs, response_model=GeneratedName)
print(response.model_dump_json(indent=2))

{
  "name": "Eiravyn",
  "culture": "Elvish",
  "meaning": "Elf of the sea"
}
